In [2]:
import os
api_key=os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

/home/zigron/anaconda3/envs/ragenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Dentistry"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract the content
content = soup.find('div', {'id': 'bodyContent'}).get_text()
paragraphs = content.split('\n\n') # chunking
paragraphs

['',
 '',
 'From Wikipedia, the free encyclopedia',
 '\nBranch of medicine',
 '\nThis article contains too many pictures for its overall length. Relevant discussion may be found on the talk page. Please improve this article by removing indiscriminate collections of images or adjusting images that are sandwiching text in accordance with the Manual of Style on use of images. (Learn how and when to remove this message)\nDentistryA dentist treats a patient with the help of a dental assistant.OccupationNames\nDentist\nDental Surgeon\nDoctor\n[1][nb 1]',
 'Occupation typeProfessionActivity sectorsHealth care, Anatomy, Physiology, Pathology, Medicine, Pharmacology, SurgeryDescriptionCompetencies\nSub-Millimeter Surgical Dexterity\nKnowledge of human health, disease, pathology, and anatomy\nCommunication/Interpersonal Skills\nAnalytical Skills\nCritical Thinking\nEmpathy/Professionalism\nEducation requiredDental DegreeFields ofemploymentPrivate practicesPrimary care clinicsHospitalsRelated job

In [6]:
def cleaning(x):
  x=x.replace('\n',' ').replace('^',' ').replace('[edit]',' ')
  x=' '.join(x.split())
  return x

new_cleaned_text=[]
for i in paragraphs:
  if i == '\nReferences[edit]':
    break
  if i == '':
    paragraphs.remove(i)
  else:
    i=cleaning(i)
    new_cleaned_text.append(i)
del(new_cleaned_text[0])
new_cleaned_text

['Branch of medicine',
 'This article contains too many pictures for its overall length. Relevant discussion may be found on the talk page. Please improve this article by removing indiscriminate collections of images or adjusting images that are sandwiching text in accordance with the Manual of Style on use of images. (Learn how and when to remove this message) DentistryA dentist treats a patient with the help of a dental assistant.OccupationNames Dentist Dental Surgeon Doctor [1][nb 1]',
 'Occupation typeProfessionActivity sectorsHealth care, Anatomy, Physiology, Pathology, Medicine, Pharmacology, SurgeryDescriptionCompetencies Sub-Millimeter Surgical Dexterity Knowledge of human health, disease, pathology, and anatomy Communication/Interpersonal Skills Analytical Skills Critical Thinking Empathy/Professionalism Education requiredDental DegreeFields ofemploymentPrivate practicesPrimary care clinicsHospitalsRelated jobsPhysiciandental assistantdental techniciandental hygienistvarious d

In [7]:
new_cleaned_text=''.join(new_cleaned_text)
new_cleaned_text

'Branch of medicineThis article contains too many pictures for its overall length. Relevant discussion may be found on the talk page. Please improve this article by removing indiscriminate collections of images or adjusting images that are sandwiching text in accordance with the Manual of Style on use of images. (Learn how and when to remove this message) DentistryA dentist treats a patient with the help of a dental assistant.OccupationNames Dentist Dental Surgeon Doctor [1][nb 1]Occupation typeProfessionActivity sectorsHealth care, Anatomy, Physiology, Pathology, Medicine, Pharmacology, SurgeryDescriptionCompetencies Sub-Millimeter Surgical Dexterity Knowledge of human health, disease, pathology, and anatomy Communication/Interpersonal Skills Analytical Skills Critical Thinking Empathy/Professionalism Education requiredDental DegreeFields ofemploymentPrivate practicesPrimary care clinicsHospitalsRelated jobsPhysiciandental assistantdental techniciandental hygienistvarious dental speci

In [8]:
# import bs4
# from langchain_community.document_loaders import WebBaseLoader
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()
# docs

In [9]:
# def chunk_string(text, chunk_size):
#   """
#   This function chunks a string into smaller pieces of a specified size.

#   Args:
#       text: The string to be chunked.
#       chunk_size: The desired size of each chunk.

#   Returns:
#       A list of strings, where each element is a chunk of the original text.
#   """
#   chunks = []
#   start_index = 0
#   while start_index < len(text):
#     end_index = min(start_index + chunk_size, len(text))
#     chunk = text[start_index:end_index]
#     chunks.append(chunk)
#     start_index = end_index

#   return chunks

# # Example usage
# text = docs
# chunk_size = 1000
# chunks = chunk_string(text, chunk_size)

# print(chunks)


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_text(new_cleaned_text)
vectorstore = Chroma.from_texts(texts=new_cleaned_text, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7dfbc43a92d0>)

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

In [12]:
system_prompt = (
    """Act as a dentist. Whenever someone will ask you about dentistry, provide the specific and to the point information. you dont have to create long passages. If you don't know the answer or it is irrelevant to dentisty, just say that you don't know or this is out of my scope ask me about desntistry related queries, don't try to make up an answer.
  you also have to same the previous responses you generated. If use ask you something about generated responses, you have to assist them.
  you have to behave like an assistant.
  "Given a chat history and the latest user question "
  "which might reference context in the chat history, "
  "formulate a standalone question which can be understood "
  "without the chat history. Do NOT answer the question, "
  "just reformulate it if needed and otherwise return it as is."
  CONTEXT: {context}"""
)

In [13]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)


In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
while True:


    question = input()
    if question=='exit':
        break
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history,'context':retriever})
    chat_history.extend(
     [
         HumanMessage(content=question),
         AIMessage(content=ai_msg["answer"]),
        ]
    )

    print(ai_msg["answer"])

A dentist is a healthcare professional who specializes in diagnosing, preventing, and treating oral health issues.
A dentist is a healthcare professional who specializes in diagnosing, preventing, and treating oral health issues.
A dentist is a licensed dental professional who specializes in oral health and hygiene, including diagnosing and treating issues related to the teeth and gums.
